## Python Basic
### Dedy Leonardo Nadeak, 12332678

I am working on developing an efficient partial least square regression (PLSR) model to predict crude protein content in wheat kernel samples using Near Infrared Spectroscopy (NIRS) data. The dataset consists of four sheets: two for calibration purposes and the remaining two for model validation. This datasets is freely available online and was compiled by Wenya Liu, 2016. Here are the three main steps I did:
1. Data separation
    - Created an argparse input that can separate single workbook based on its sheets.
2. Data preprocessing and spectra plotting
    - Created an argparse input that can
        a. remove sample data that contain null value
        b. process preprocessing data, such as Multiplicative scatter correction (MSC), Standard Normal Variate (SNV), first and second derivative.
        c. plot all spectra, before and after pretreatment
        d. export the plot as an image file
3. Build PLSR models and determine the most effective one.
    - Create functions that can use to fit a model and plot it.
    - By comparing the performance of different models, I determined the most effective one.
    
Source:
Wenya, L. (2016). Wheat kernel dataset: Figshare. Retrieved from https://figshare.com/articles/wheat_kernel_dataset/4252217/1

#### Data Import
Datasetes is separated based on number of sheets

In [1]:
%%writefile split_excel_by_sheets.py
"""
    Reads an input Excel file and generates separate output CSV files for each sheet.

    Actions:
        input_excel_path (str): Path to the input Excel file.
        output_folder (str): Path to the folder where output CSV files will be saved.
"""
import pandas as pd
import os

def split_excel_by_sheets(input_excel_path, output_folder):

    # Read the input Excel file into a dictionary of DataFrames (one for each sheet)
    excel_file = pd.ExcelFile(input_excel_path)
    sheet_data = {sheet_name: excel_file.parse(sheet_name) for sheet_name in excel_file.sheet_names}

    # Create separate output CSV files for each sheet
    os.makedirs(output_folder, exist_ok=True)
    for sheet_name, df in sheet_data.items():
        output_csv_path = os.path.join(output_folder, f"{sheet_name}.csv")
        df.to_csv(output_csv_path, index=False)
        print(f"Saved {sheet_name}.csv")
    
if __name__ == '__main__':
    import argparse
    parser = argparse.ArgumentParser(description='Plotting, Preprocessing and Derivate the NIR Spectra')
    parser.add_argument('-i', '--input_excel', type=str, help='Excel file location', required = True)
    parser.add_argument('-o', '--out_folder', default = "output_csv", type=str, help='Output folder location')
    args = parser.parse_args()
    
    split_excel_by_sheets(args.input_excel, args.out_folder)

Overwriting split_excel_by_sheets.py


In [2]:
!python split_excel_by_sheets.py -i wheatkernel.xlsx

Saved calibration_X.csv
Saved calibration_Y.csv
Saved test_X.csv
Saved test_Y.csv


#### Data Preprocessing and spectra plotting
in NIRS datasets, data preprocessing plays a crucial role. Applying techniques like SNV or MSC helps remove scatter data caused by physical effects, leaving behind only the chemical effects. Additionally, plotting the absorbance data againts their wavelength allows us to evaluate which chemical functional groups are reponsible to the final model.

In [3]:
import matplotlib.pyplot as plt
import numpy as np

# Wenya stated that the NIR spectra was recorded at the region 850-1050 nm
# Create a variable named wavelength to be used as x-axis in the spectra plotting
wavelength = np.array(list(range(850,1050,2)))

#export as a csv file
np.savetxt('wavelength.csv', wavelength, delimiter=',', fmt='%d')

In [4]:
%%writefile nir_plot_der.py
"""
    This script processes Near-Infrared (NIR) spectra data from a CSV file, performs various preprocessing steps,
    and generates plots of the original, preprocessed, and derivative spectra.

    Functions:
    - remove_na: Removes rows with NA or null values from the dataframe.
    - combine_wavelength: Combines wavelength data with the absorbance values if they are in separate CSV files.
    - plotting: Plots the spectra data.
    - preprocess_msc: Applies Multiplicative Scatter Correction (MSC) preprocessing to the spectra.
    - preprocess_snv: Applies Standard Normal Variate (SNV) preprocessing to the spectra.
    - derivate: Calculates the derivative of the spectra using Savitzky-Golay smoothing.

    Usage:
    The script can be run from the command line with various options for preprocessing and plotting.

    Arguments:
    - -plot: Path to the input CSV file containing the spectra data.
    - -w, --wavelength: (Optional) Path to the CSV file containing wavelength data.
    - -o, --out_file: (Optional) Path to save the output plot as a PNG file.
    - -msc: Apply MSC preprocessing (mutually exclusive with SNV).
    - -snv: Apply SNV preprocessing (mutually exclusive with MSC).
    - -der: Apply derivative calculation. Choices are 'der1' for 1st derivative and 'der2' for 2nd derivative.

    Example usage:
    python nir_plot_der.py -plot spectra.csv -w wavelength.csv -o output.png -msc -der der1 der2

    The script generates and saves or displays plots of the original, preprocessed, and derivative spectra.
"""
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from chemotools.derivative import SavitzkyGolay

#cleaning NA data
def remove_na(df):
    if df.isnull().values.any():
        print("NA or null values found. Removing rows with NA or null values")
        df_cleaned = df.dropna()
    else:
        df_cleaned = df
    return df_cleaned

#Combine wavelength with the absorbance if csv doesn't contain the wavelength
def combine_wavelength(df, wavelength):
    wave_data = wavelength.values.flatten()
    df_data = df.values
    
    # Create a DataFrame from df_data
    combined_df = pd.DataFrame(df_data, columns=wave_data)

    return combined_df

#Plotting
def plotting(df):
    x = df.columns.values
    y = df.iloc[0:].values
    y_transpose = y.T
    plt.plot(x, y_transpose)
    plt.xlabel("Wavelength (nm)")
    plt.ylabel("Absorption") 
        

# Multiplicative scatter correction (MSC) preprocessing
def preprocess_msc(df):
    # Convert DataFrame to numpy array
    y = df.iloc[0:].values
    
    # Zero mean each spectrum (subtract the mean of each row)
    y_centered = y - np.mean(y, axis=1, keepdims=True)
    
    # Initialize variables
    temp = []
    y_msc = np.zeros(y.shape)
    
    # Process each spectrum
    for i in range(y_centered.shape[0]):
        for j in range(0, y_centered.shape[0], 10):
            temp.append(np.mean(y_centered[j:j + 10], axis=0))
        
        # Compute the mean of the temp array for polyfit
        mean_temp = np.mean(temp, axis=0)
        fit = np.polyfit(mean_temp, y_centered[i, :], 1)
        y_msc[i, :] = (y_centered[i, :] - fit[1]) / fit[0]
    
    # Convert numpy array back to DataFrame
    df_msc = pd.DataFrame(y_msc, columns=df.columns, index=df.index)
    
    return df_msc

#Standard Normal Variate (SNV) preprocessing
def preprocess_snv(df):
    # Convert DataFrame to numpy array
    y = df.iloc[0:].values
    
    #initialize variable
    y_snv = np.zeros_like(df)
    
    #running the correction
    for i in range(y.shape[0]):
        y_snv[i,:] = (y[i,:] - np.mean(y[i,:])) / np.std(y[i,:])
        
    # Convert numpy array back to dataframe
    df_snv = pd.DataFrame(y_snv, columns=df.columns, index=df.index)
    return df_snv

#derivatization
def derivate(df, window=15, p_order=2, d_order=1):
    sg = SavitzkyGolay(window_size=window, polynomial_order=p_order, derivate_order=d_order)
    # Convert DataFrame to numpy array
    y = df.iloc[0:].values
    der = sg.fit_transform(y)
    df_der = pd.DataFrame(der, columns=df.columns, index=df.index)
    return df_der

def main():
    import argparse
    parser = argparse.ArgumentParser(description='Plotting, Preprocessing and Derivate the NIR Spectra')
    
    #Plotting
    parser.add_argument('-plot', help="Plotting the NIR spectra and output as a jpg", type=str, required = True)
    parser.add_argument('-w','--wavelength', help="Reading a csv file that contains wavelength of the spectra", type=str)
    parser.add_argument('-o', '--out_file', type=str, help='Output PNG file for the plot')
    
    #Mutually exclusive group for MSC and SNV options
    group = parser.add_mutually_exclusive_group(required = True)
    group.add_argument('-msc', help="Input must be a dataframe to run MSC function", action = 'store_true')
    group.add_argument('-snv', help="Input must be a dataframe to run SNV function", action = 'store_true')
    
    #Derivative can run after choosing SNV or MSC preprocessing
    parser.add_argument('-der', nargs='+', help="Input must be a dataframe to run derivative function", choices=['der1', 'der2'])
    
    args = parser.parse_args()
    
     # Reading the data frame
    imported_df = pd.read_csv(args.plot, header=None)
    imported_df = remove_na(imported_df)
    if args.wavelength:
        wavelength = pd.read_csv(args.wavelength, header=None)
        df = combine_wavelength(imported_df, wavelength)
    else:
        df = imported_df
    
    # Determine number of plots to create
    num_plots = 1
    if args.msc or args.snv:
        num_plots += 1  
    if args.der:
        num_plots += len(args.der)
    
    # Create a new figure
    plt.figure(figsize=(12, 4))
    
    # Plotting
    plot_index = 1
    
    # Plot original data
    plt.subplot(1, num_plots, plot_index)
    plotting(df)
    plt.title('Original Data')
    plot_index += 1
    
    #preprocessing execution
    if args.msc:
        preprocess_df = preprocess_msc(df)
        plt.subplot(1, num_plots, plot_index)
        plotting(preprocess_df)
        plt.title('MSC Preprocessed Data')
        plot_index += 1
    elif args.snv:
        preprocess_df = preprocess_snv(df)
        plt.subplot(1, num_plots, plot_index)
        plotting(preprocess_df)
        plt.title('SNV Preprocessed Data')
        plot_index += 1
    
    #derivative execution
    if args.der:
        for der_option in args.der:
            if der_option == "der1":
                der1 = derivate(preprocess_df, d_order=1)
                plt.subplot(1, num_plots, plot_index)
                plotting(der1)
                plt.title('1st Derivative')
                plot_index += 1
            elif der_option == "der2":
                der2 = derivate(preprocess_df, d_order=2)
                plt.subplot(1, num_plots, plot_index)
                plotting(der2)
                plt.title('2nd Derivative')
                plot_index += 1

    # Check if the output location is available
    if args.out_file:
        #output path
        os.makedirs("output_plot", exist_ok=True)
        output_plot_path = os.path.join("output_plot", args.out_file)
        #export the plot
        plt.savefig(output_plot_path)
        print(f"Plot saved as {args.out_file}")
    else:
        plt.tight_layout()
        plt.show()
if __name__ == '__main__':
    main()    

Overwriting nir_plot_der.py


In [5]:
#Plotting and export the output
!python nir_plot_der.py -plot output_csv/calibration_X.csv -w wavelength.csv -o plot.png -snv

Plot saved as plot.png


In [6]:
#Plotting without exporting the output
!python nir_plot_der.py -plot output_csv/calibration_X.csv -w wavelength.csv -msc -der der1 der2

#### Build PLSR models and determine the most effective one.
To construct Partial Least Squares Regression (PLSR) models and identify the most effective one, I utilized two datasets: one for model calibration and another for model evaluation. Specifically, I built six PLSR models, with two of them employing Standard Normal Variate (SNV) and Multiplicative Scatter Correction (MSC) preprocessing without derivatives, while the remaining models incorporated first and second derivatives. The evaluation criteria included coefficient correlation (R²), mean square error (MSE), and the number of components. To enhance efficiency, I restricted the use of ten variables as the maximum number of components in each model. 

In [7]:
"""This script is used to create PLS model from the NIR spectra"""
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.cross_decomposition import PLSRegression

def fitting(cal_x, cal_y, test_x, test_y, n_comp = 2):
    model = PLSRegression(n_components = n_comp)
    model.fit(cal_x, cal_y)
    
    pred_y = model.predict(test_x)
    mse = metrics.mean_squared_error(test_y, pred_y)
    r2 = metrics.r2_score(test_y, pred_y)
    
    return mse, r2, pred_y

def best_fit(cal_x, cal_y, test_x, test_y, n_comp = 2):
    #initialize
    best_n_comp = 0
    best_mse = 0
    best_r2 = 0
    best_pred_y = []
    total_mse = []
    total_r2 = []
    total_ncomp = []
    #iteration for fitting
    for i in range(2, n_comp + 1):
        mse, r2, pred_y = fitting(cal_x, cal_y, test_x, test_y, n_comp = i)
        total_mse.append(mse)
        total_r2.append(r2)
        total_ncomp.append(i)
        if r2 > best_r2:
            best_r2 = r2
            best_mse = mse
            best_n_comp = i
            best_pred_y = pred_y
    # MSE plotting
    plt.subplot(131)
    plt.title("Mean Square Error")
    plt.plot(total_ncomp, total_mse, marker='.')
    plt.scatter([best_n_comp], [best_mse], color='red', zorder=5)  # Highlight the lowest MSE
    plt.xlabel('Number of Components')
    plt.ylabel('MSE')
    plt.xticks(range(2, n_comp + 1))

    # R2 to number of component Plotting
    plt.subplot(132)
    plt.title(f"$R^2$ Score")
    plt.plot(total_ncomp, total_r2, marker='.')
    plt.scatter([best_n_comp], [best_r2], color='red', zorder=5)  # Highlight the highest R2
    plt.xlabel('Number of Components')
    plt.ylabel(f"$R^2$")
    plt.xticks(range(2, n_comp + 1))
    
    # Prediction vs Test
    plt.subplot(133)
    plt.title(f"Prediction Performance (ncomp = {best_n_comp})")
    plt.scatter(test_y, best_pred_y)
    plt.xlabel('Reference values')
    plt.ylabel('Prediction')
    plt.text(0.05, 0.95, f'Max $R^2$: {best_r2:.2f}', transform=plt.gca().transAxes,
             verticalalignment='top', bbox=dict(facecolor='white', alpha=0.5))
    
    plt.show()
    return best_r2, best_mse, best_n_comp

In [8]:
#Import all data are needed
import pandas as pd
import nir_plot_der as npd

xcal = pd.read_csv("output_csv/calibration_X.csv", header=None)
ycal = pd.read_csv("output_csv/calibration_Y.csv", header=None)
xtest = pd.read_csv("output_csv/test_X.csv", header=None)
ytest = pd.read_csv("output_csv/test_Y.csv", header=None)

#preprocessing xcal and xtest
xcal_snv_0 = npd.preprocess_snv(xcal).copy()
xtest_snv_0 = npd.preprocess_snv(xtest).copy()

In [9]:
##Fitting for SNV preprocessing and without derivative.
r2_snv_0, mse_snv_0, comp_snv_0 = best_fit(xcal_snv_0, ycal, xtest_snv_0, ytest, n_comp = 10)

print(f"The best fitting SNV model without derivatives produces an R² value of {round(r2_snv_0, 2)} and an MSE of {round(mse_snv_0, 2)} with 8 components.")

The best fitting SNV model without derivatives produces an R² value of 0.89 and an MSE of 0.33 with 8 components.


In [10]:
#Prepare for MSC preprocessing and without derivative.
xcal_msc_0 = npd.preprocess_msc(xcal).copy()
xtest_msc_0 = npd.preprocess_msc(xtest).copy()

##Fitting for MSC preprocessing and without derivative.
r2_msc_0, mse_msc_0, comp_msc_0 = best_fit(xcal_msc_0, ycal, xtest_msc_0, ytest, n_comp = 10)

In [11]:
#Prepare for SNV preprocessing and with first derivative.
xcal_snv_1 = npd.derivate(xcal_snv_0, d_order = 1).copy()
xtest_snv_1 = npd.derivate(xtest_snv_0, d_order = 1).copy()

##Fitting for MSC preprocessing and without derivative.
r2_snv_1, mse_snv_1, comp_snv_1 = best_fit(xcal_snv_1, ycal, xtest_snv_1, ytest, n_comp = 10)

In [12]:
#Prepare for SNV preprocessing and with second derivative.
xcal_snv_2 = npd.derivate(xcal_snv_0, d_order = 2).copy()
xtest_snv_2 = npd.derivate(xtest_snv_0, d_order = 2).copy()

##Fitting for MSC preprocessing and without derivative.
r2_snv_2, mse_snv_2, comp_snv_2 = best_fit(xcal_snv_2, ycal, xtest_snv_2, ytest, n_comp = 10)

In [13]:
#Prepare for MSC preprocessing and with first derivative.
xcal_msc_1 = npd.derivate(xcal_msc_0, d_order = 1).copy()
xtest_msc_1 = npd.derivate(xtest_msc_0, d_order = 1).copy()

##Fitting for MSC preprocessing and without derivative.
r2_msc_1, mse_msc_1, comp_msc_1 = best_fit(xcal_msc_1, ycal, xtest_msc_1, ytest, n_comp = 10)

In [14]:
#Prepare for MSC preprocessing and with first derivative.
xcal_msc_2 = npd.derivate(xcal_msc_0, d_order = 2).copy()
xtest_msc_2 = npd.derivate(xtest_msc_0, d_order = 2).copy()

##Fitting for MSC preprocessing and without derivative.
r2_msc_2, mse_msc_2, comp_msc_2 = best_fit(xcal_msc_2, ycal, xtest_msc_2, ytest, n_comp = 10)

In [15]:
#Summary of the model
import pandas as pd

# Create data for the summary table
data = {
    'Preprocessing': ['SNV', 'SNV', 'SNV', 'MSC', 'MSC', 'MSC'],
    'Derivative': [0, 1, 2, 0, 1, 2],
    'R²': [round(r2_snv_0, 2), round(r2_snv_1, 2), round(r2_snv_2, 2), round(r2_msc_0, 2), round(r2_msc_1, 2), round(r2_msc_2, 2)],
    'MSE': [round(mse_snv_0, 2), round(mse_snv_1, 2), round(mse_snv_2, 2), round(mse_msc_0, 2), round(mse_msc_1, 2), round(mse_msc_2, 2)],
    'Number of Components': [round(comp_snv_0, 2), round(comp_snv_1, 2), round(comp_snv_2, 2), round(comp_msc_0, 2), round(comp_msc_1, 2), round(comp_msc_2, 2)]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)


  Preprocessing  Derivative    R²   MSE  Number of Components
0           SNV           0  0.89  0.33                     8
1           SNV           1  0.88  0.36                     7
2           SNV           2  0.92  0.26                     6
3           MSC           0  0.30  2.11                     3
4           MSC           1  0.79  0.64                     7
5           MSC           2  0.85  0.45                     7


## Conclusion
The most effective model is the third model with SNV preprocessing and second derivative. This model has the highest R² values and the lowest MSE with only six number of components.